In [1]:
training_data = [
    ['Green', 3, 'Mango'],
    ['Yellow', 3, 'Mango'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon']
]

head = ["color", "diameter", "label"]

# Find the unique values for a column in a dataset.
def unique_vals(rows, col):
    return set([row[col] for row in rows])

#######
# Demo:
# unique_vals(training_data, 0)
# unique_vals(training_data, 1)
#######


# Count the number of each type of example in a dataset.
def class_counts(rows):
    counts = {}
    for row in rows:
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

#######
# Demo:
# class_counts(training_data)
#######


# Test if a value is numeric.
def is_numeric(value):
    return isinstance(value, int) or isinstance(value, float)

#######
# Demo:
# is_numeric(7)
# is_numeric("Red")
#######


# A Question is used to partition a dataset.
# This class just records a 'column number' (e.g., 0 for Color) and a 'column value' (e.g., Green).
# The 'match' method is used to compare the feature value in an example to the feature value stored in the question.
class Question:
    def __init__(self, column, value):
        self.column = column
        self.value = value
        
    # Compare the feature value in an example to the feature value in this question.
    def match(self, example):
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value
    
    # This is just a helper method to print the question in a readable format.
    def __repr__(self):
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            head[self.column], condition, str(self.value))
    
# Partitions a dataset.
# For each row in the dataset, check if it matches the question.
# If so, add it to 'true rows'. Otherwise, add it to 'false rows'.
def partition(rows, question):
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

#######
# Demo:
# Let's partition the training data based on whether rows are Red.
# true_rows, false_rows = partition(training_data, Question(0, 'Red'))
# true_rows will contain all the 'Red' rows.
# false_rows will contain everything else.


# Calculate the Gini Impurity for a list of rows.
def gini(rows):
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity


# Information Gain: the uncertainty of the starting node minus the weighted impurity of two child nodes.
def info_gain(left, right, current_uncertainty):
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)


# Find the best question to ask by iterating over every feature / value and calculating the information gain.
def find_best_split(rows):
    best_gain = 0  # keep track of the best information gain
    best_question = None  # keep track of the feature / value produced it
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1  # number of column
    
    for col in range(n_features):  # for each feature
        values = set([row[col] for row in rows])  # unique values in the column
        for val in values:  # for each value
            question = Question(col, val)
            
            # try splitting the dataset
            true_rows, false_rows = partition(rows, question)
            
            # skip this split if it doesn't divide the dataset
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue
            
            # Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)
            
            if gain >= best_gain:
                best_gain, best_question = gain, question
                
    return best_gain, best_question


# A Leaf node classifies data.
# This holds a dictionary of class (e.g., Mango) -> number of times it appears in the rows from
# the training data that reach the leaf.
class Leaf:
    def __init__(self, rows):
        self.predictions = class_counts(rows)
        
        
# A Decision node ask a question.
# This holds a reference to the question, and to the two child nodes.
class Decision_Node:
    def __init__(self, question, true_branch, false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

        
# Build the tree
def build_tree(rows):
    # Try partitioning the dataset on each of the unique attribute, calculate the information gain,
    # and return the question that produces the highest gain.
    gain, question = find_best_split(rows)
    
    # Base case: no further info gain
    # Since we can ask no further questions, we'll return a leaf.
    if gain == 0:
        return Leaf(rows)
    
    # If we reach here, we have found a useful feature / value to partition on.
    true_rows, false_rows = partition(rows, question)
    
    # Recursively build the true branch.
    true_branch = build_tree(true_rows)
    # Recursively build the false branch.
    false_branch = build_tree(false_rows)
    
    # Return a Question node.
    # This records the best feature / value to ask at this point,
    # as well as the branch to follow depending on the answer.
    return Decision_Node(question, true_branch, false_branch)


def print_tree(node, spacing=""):
    #Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print(spacing + "Predict", node.predictions)
        return
    
    # Print the question at this node
    print(spacing + str(node.question))
    
    # Call this function recursively on the true branch
    print(spacing + '--> True:')
    print_tree(node.true_branch, spacing = "  ")
    
    # Call this function recursively on the false branch
    print(spacing + '--> False:')
    print_tree(node.false_branch, spacing = "  ")
    

def classify(row, node):
    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        return node.predictions
    
    # Decide whether to follow the true branch or the false branch.
    # Compare the feature / value stored in the node to the example we are considering.
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)
    

# Print the predictions at a leaf.
def print_leaf(counts):
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs


if __name__ == '__main__':
    my_tree = build_tree(training_data)
    
    print_tree(my_tree)
    
    # Evaluate
    testing_data = [
        ['Green', 3, 'Mango'],
        ['Yellow', 4, 'Mango'],
        ['Red', 2, 'Grape'],
        ['Red', 1, 'Grape'],
        ['Yellow', 3, 'Lemon']
    ]
    
    for row in testing_data:
        print("Actual: %s. Predicted: %s" % 
             (row[-1], print_leaf(classify(row, my_tree))))

Is diameter >= 3?
--> True:
  Is color == Yellow?
  --> True:
  Predict {'Mango': 1, 'Lemon': 1}
  --> False:
  Predict {'Mango': 1}
--> False:
  Predict {'Grape': 2}
Actual: Mango. Predicted: {'Mango': '100%'}
Actual: Mango. Predicted: {'Mango': '50%', 'Lemon': '50%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Lemon. Predicted: {'Mango': '50%', 'Lemon': '50%'}
